In [ ]:
import torch 
from transformers import BertTokenizer, BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model.load_state_dict(torch.load('../models/checkpoints/bert_20.07_13:15.pth', map_location='cpu'))
model.eval()  

In [2]:
inputs = tokenizer(
    'heyy',
    'hello',
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)
inputs = {k: v.to('cpu') for k, v in inputs.items()}
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    probs = logits.softmax(1)

/home/marinda/Документы/base/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [3]:
inputs

{'input_ids': tensor([[ 101, 4931, 2100,  102, 7592,  102,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
      

In [4]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.1155, -0.2844]]), hidden_states=None, attentions=None)

In [ ]:
batch_size = 1
max_length = 128  

dummy_input = {
    'input_ids': torch.randint(0, 1000, (batch_size, max_length), dtype=torch.long),
    'token_type_ids': torch.zeros(batch_size, max_length, dtype=torch.long),
    'attention_mask': torch.ones(batch_size, max_length, dtype=torch.long)
}

torch.onnx.export(
    model,                         
    (dummy_input['input_ids'], 
     dummy_input['attention_mask'], 
     dummy_input['token_type_ids']), 
    "model.onnx",                   
    export_params=True,           
    opset_version=14,             
    do_constant_folding=True,      
    input_names=['input_ids', 'attention_mask', 'token_type_ids'], 
    output_names=['logits'],          
    dynamic_axes={
        'input_ids': {0: 'batch_size', 1: 'sequence'},
        'attention_mask': {0: 'batch_size', 1: 'sequence'},
        'token_type_ids': {0: 'batch_size', 1: 'sequence'},
        'last_hidden_state': {0: 'batch_size', 1: 'sequence'},
        'pooler_output': {0: 'batch_size'}
    }
)